# Example-1-Tabular training

How to use the tabular application in fastai. To illustrate the tabular application, we will use the example of the [Adult dataset](https://archive.ics.uci.edu/ml/datasets/Adult) where we have to predict if a person is earning more or less than $50k per year using some general data.

In [29]:
from fastai.tabular.all import *

### Load data

In [36]:
path = '/Users/rakesh_poduval/Projects/Py-Modules/Fast-AI/Example-44-tabular.ipynb'
df = pd.read_csv(path+'/adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


###  Data pre-processing 

Some of the columns are continuous (like age) and we will treat them as float numbers we can feed our model directly. Others are categorical (like workclass or education) and we will convert them to a unique index that we will feed to embedding layers. We can specify our categorical and continuous column names, as well as the name of the dependent variable in `TabularDataLoaders` factory methods:

In [37]:
dls = TabularDataLoaders.from_csv(
    path+'/adult.csv',
    # path=path,
    y_names="salary",
    cat_names = ['workclass','education','marital-status','occupation','relationship','race'],
    cont_names = ['age','fnlwgt','education-num'],
    procs = [Categorify, FillMissing, Normalize])
dls

The last part is the list of pre-processors we apply to our data:

- `Categorify` is going to take every categorical variable and make a map from integer to unique categories, then replace the values by the corresponding index.
- `FillMissing` will fill the missing values in the continuous variables by the median of existing values (you can choose a specific value if you prefer)
- `Normalize` will normalize the continuous variables (substract the mean and divide by the std)

To further expose what's going on below the surface, let's rewrite this utilizing `fastai`'s `TabularPandas` class. We will need to make one adjustment, which is defining how we want to split our data. By default the factory method above used a random 80/20 split, so we will do the same:

Once we build our `TabularPandas` object, our data is completely preprocessed as seen below:

In [32]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))
to = TabularPandas(
    df, 
    procs=[Categorify,FillMissing,Normalize],
    cat_names = ['workclass','education','marital-status','occupation','relationship','race'],
    cont_names = ['age','fnlwgt','education-num'],
    y_names='salary',
    splits=splits)

to.xs.iloc[:2]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
27905,5,10,3,2,1,5,1,-0.408179,-0.082314,1.142471
29043,5,10,1,13,2,3,1,0.691288,-0.383423,1.142471


Now we can build our `DataLoaders` again. Later we will explore why using `TabularPandas` to preprocess will be valuable. The `show_batch` method works like for every other application:

In [39]:
dls = to.dataloaders(bs=64)
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,90.0,175490.999308,9.0,>=50k
1,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,False,52.0,180880.999677,13.0,<50k
2,Private,HS-grad,Divorced,Other-service,Not-in-family,White,False,59.0,212783.000702,9.0,<50k
3,Private,Bachelors,Separated,Sales,Own-child,Black,False,31.0,309973.994060,13.0,<50k
4,Private,HS-grad,Never-married,Craft-repair,Not-in-family,White,False,25.0,234056.999868,9.0,<50k
5,Private,Doctorate,Married-civ-spouse,Exec-managerial,Husband,White,False,46.0,188386.000031,16.0,>=50k
6,Private,Doctorate,Married-civ-spouse,Prof-specialty,Husband,White,False,42.0,146658.999822,16.0,>=50k
7,Private,Some-college,Divorced,Adm-clerical,Not-in-family,White,False,54.0,220115.000884,10.0,<50k
8,Private,7th-8th,Never-married,Handlers-cleaners,Not-in-family,Black,False,42.0,242563.998824,4.0,<50k
9,Private,HS-grad,Never-married,Protective-serv,Own-child,White,False,30.0,425528.003187,9.0,<50k


# Model building

We can define a model using the `tabular_learner` method. When we define our model, `fastai` will try to infer the loss function based on our `y_names` earlier. And we can train that model with the `fit_one_cycle` method (the `fine_tune` method won't be useful here since we don't have a pretrained model).
> Sometimes with tabular data, your `y`'s may be encoded (such as 0 and 1). In such a case you should explicitly pass `y_block = CategoryBlock` in your constructor so `fastai` won't presume you are doing regression.

In [51]:
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,accuracy,time
0,0.372924,0.382128,0.821560,00:03
1,0.354389,0.366512,0.828163,00:03


We can then have a look at some predictions:

In [52]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5.0,10.0,5.0,14.0,2.0,5.0,1.0,-0.701370,-0.300961,1.142471,0.0,0.0
1,3.0,16.0,5.0,0.0,4.0,5.0,2.0,-0.701370,0.004946,-0.032600,0.0,0.0
2,5.0,12.0,5.0,13.0,2.0,5.0,1.0,-0.481477,0.000299,-0.424290,0.0,0.0
3,6.0,16.0,3.0,5.0,1.0,2.0,1.0,0.104906,-0.495277,-0.032600,0.0,1.0
4,5.0,10.0,1.0,9.0,2.0,5.0,1.0,1.277670,-0.646441,1.142471,0.0,0.0
5,5.0,12.0,5.0,8.0,4.0,5.0,1.0,-1.434348,-0.261737,-0.424290,0.0,0.0
6,5.0,13.0,3.0,11.0,1.0,4.0,1.0,-0.188285,-1.298459,1.534161,1.0,1.0
7,8.0,12.0,3.0,6.0,1.0,5.0,1.0,-0.554774,-0.303492,-0.424290,0.0,0.0
8,5.0,12.0,5.0,8.0,2.0,5.0,1.0,-1.287752,0.492494,-0.424290,0.0,0.0


Or use the predict method on a row:

In [53]:
row, clas, probs = learn.predict(df.iloc[0])
print(clas, probs)

row.show()

tensor(1) tensor([0.4725, 0.5275])


,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101320.000354,12.0,>=50k


To get prediction on a new dataframe, you can use the `test_dl` method of the `DataLoaders`. That dataframe does not need to have the dependent variable in its column. Then `Learner.get_preds` will give you the predictions:
> Since machine learning models can't magically understand categories it was never trained on, the data should reflect this. If there are different missing values in your test data you should address this before training

In [49]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)
learn.get_preds(dl=dl)

## `fastai` with Other Libraries

As mentioned earlier, `TabularPandas` is a powerful and easy preprocessing tool for tabular data. Integration with libraries such as Random Forests and XGBoost requires only one extra step, that the `.dataloaders` call did for us. Let's look at our `to` again. It's values are stored in a `DataFrame` like object, where we can extract the `cats`, `conts,` `xs` and `ys` if we want to:

In [ ]:
to.xs[:3]

> Now that everything is encoded, you can then send this off to XGBoost or Random Forests by extracting the train and validation sets and their values:

In [54]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

And now we can directly send this in!